In [83]:
import numpy as np
import pandas as pd
from pomegranate import *
import random
import os

In [19]:
df = pd.read_csv('AdjustedRPKMOutput/RPF_chrE.txt',sep='\t')

In [20]:
df.head(5)

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,93.0,16.468792,62.0,13.189469,49.0,14.172717,37.0,9.403085,39.0,10.487249
1,NM_001143986,TLE6,4.0,0.562465,2.0,0.337849,2.0,0.459350,1.0,0.201802,1.0,0.213527
2,NM_001003803,ATP5S,81.0,30.214920,83.0,37.193868,71.0,43.258655,46.0,24.625447,26.0,14.727463
3,NM_001003800,BICD2,501.0,47.157866,389.0,43.986848,284.0,43.662942,257.0,34.716778,230.0,32.874747
4,NM_016649,ESF1,69.0,6.525288,52.0,5.907596,41.0,6.333047,26.0,3.528692,26.0,3.733723


### RPF data

In [21]:
X = np.log2(df[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']].values).astype(int)

In [22]:
print(X.shape)

(11745, 5)


In [23]:
LENGTH, DIMENSION = X.shape

In [24]:
bin_size = 20 #state number of bins here, multiple of 5
step = (np.max(X)-np.min(X))/bin_size
bins = np.arange(np.min(X),np.max(X)+0.1,step)
print(bins)
X = np.digitize(X,bins)

[-5.  -4.1 -3.2 -2.3 -1.4 -0.5  0.4  1.3  2.2  3.1  4.   4.9  5.8  6.7
  7.6  8.5  9.4 10.3 11.2 12.1 13. ]


In [25]:
print(X[:5])

[[10  9  9  9  9]
 [ 6  5  5  4  4]
 [10 12 12 10  9]
 [12 12 12 12 12]
 [ 8  8  8  7  7]]


### Sample network

In [26]:
model = BayesianNetwork.from_samples(X,algorithm='chow-liu')

In [27]:
model.structure

((), (0,), (1,), (2,), (3,))

### Initialize 4 networks

In [96]:
K = 6

models = []
x_k = [[] for i in range(K)] #initialize K empty data arrays
alpha_k = []
indices_array = [[] for i in range(K)]
print("**************** K =", K ,"************************")


# Sequences for initial CL Multinet Estimation
# Make K subsets of data
for i in range(LENGTH):
    random_integer = random.randint(0,K-1)
    x_k[random_integer].append(X[i])
    indices_array[random_integer].append(i)
            
for i in range(K):
    print("Length of model",i+1,":",len(x_k[i]))
    alpha_k.append(len(x_k[i])/LENGTH)
    model = BayesianNetwork.from_samples(x_k[i],algorithm='chow-liu') 
    models.append(model)

print("Initial Model Structures",alpha_k)
for model in models:
    print(model.structure)

print("Initial Alphas",alpha_k)

CML = 0

for i in range(K):
    x = x_k[i]
    model = models[i]
    CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[i])
print("Initial CML",CML)

**************** K = 6 ************************
Length of model 1 : 1909
Length of model 2 : 1998
Length of model 3 : 2005
Length of model 4 : 1972
Length of model 5 : 1862
Length of model 6 : 1999
Initial Model Structures [0.16253724989357174, 0.17011494252873563, 0.17071094082588337, 0.16790123456790124, 0.15853554704129416, 0.17020008514261387]
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
Initial Alphas [0.16253724989357174, 0.17011494252873563, 0.17071094082588337, 0.16790123456790124, 0.15853554704129416, 0.17020008514261387]
Initial CML -87226.39428365581


### EM Step

In [97]:
init_CML = CML

#S Step
for i in range(5): #start with 100 iterations of EM
    x_k = [[] for i in range(K)] #initialize K empty data arrays
    #S step: Assign data-points randomly
    for j in range(LENGTH):
        x_k[random.randint(0,K-1)].append(X[j])
    alpha_k = [len(x_k[k])/LENGTH for k in range(K)]
    models = []
    for j in range(K):
        model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
        models.append(model)
        
    #M step: Calculate the CML criterion and re-estimate parameters
    init_CML = CML
    CML=0
    for j in range(K):
        x = x_k[j]
        model = models[j]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
    print("New CML is:", CML)   
    if(CML==init_CML): #if no change in CML, then break
        break


New CML is: -87314.02959328138
New CML is: -87292.25668097037
New CML is: -87301.68585425513
New CML is: -87250.02858219786
New CML is: -87250.06141785166


In [99]:
# C Step     
for i in range(100): #start with 100 iterations of EM
    x_k_temp = [[] for i in range(K)] #initialize K empty data arrays for C step (assign)
    indices_array = [[] for i in range(K)]
    #E Step: Calculate each point's posterior probability for K trees
    for idx_first,x in enumerate(X):
        model_prob = []
        for idx,model in enumerate(models): # K trees
            try:
                model_prob.append(model.probability(x))
            except KeyError: #if a point doesn't exist in a tree, then the probability is zero
                model_prob.append(0)
        max_prob_idx = model_prob.index(max(model_prob)) #return index of the max posterior probability
        x_k_temp[max_prob_idx].append(x)
        indices_array[max_prob_idx].append(idx_first)
    
    #C step: Assign data-points to the trees that maximize their posterior probability
    x_k = x_k_temp
    alpha_k = [len(x_k[i])/LENGTH for i in range(K)]
    models = []
    for j in range(K):
        model = BayesianNetwork.from_samples(x_k[j],algorithm='chow-liu') 
        models.append(model)
        
    #M step: Calculate the CML criterion and re-estimate parameters
    init_CML = CML
    CML=0
    for j in range(K):
        x = x_k[j]
        model = models[j]
        CML+=sum(np.log(model.probability(x)))+len(x)*np.log(alpha_k[j])
    print("New CML is:", CML)   
    if(CML==init_CML): #if no change in CML, then break
        [print(model.structure) for model in models]
        for idx,indices in enumerate(indices_array):
            genes=[]
            acc_nums=[]
            for index in indices:
                genes.append(df.iloc[index]['GeneName'])
                acc_nums.append(df.iloc[index]['AccNum'])
            print(len(indices),len(genes),len(acc_nums))
#             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array)))
#             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/Genes/')
#             os.mkdir('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/AccNums/')
            with open('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/Genes/Gene'+str(idx+1)+'.txt','w') as f:
                for gene in genes:
                    f.write("%s\n" % gene)
            with open('/Users/akankshitadash/Desktop/Bayesian Networks/'+str(len(indices_array))+'/AccNums/AccNum'+str(idx+1)+'.txt','w') as f:
                for acc_num in acc_nums:
                    f.write("%s\n" % acc_num)
        break

New CML is: -66501.17260272024
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (1,))
((), (0,), (1,), (2,), (3,))
((), (0,), (1,), (2,), (3,))
((), (0,), (4,), (1,), (3,))
((), (0,), (1,), (2,), (3,))
2115 2115 2115
1659 1659 1659
2020 2020 2020
2312 2312 2312
1623 1623 1623
2016 2016 2016
